## Exp3_Senariowise_CompvsIncomp_Push_V1

Author : @leopauly | cnlp@leeds.ac.uk <br>
Program: Tying to show that the proposed activity feature method can distinguish between complete vs Incomplete actions scenario by scenario

In [2]:
import os
from six.moves import xrange  # pylint: disable=redefined-builtin
import PIL.Image as Image
import random
import numpy as np
import cv2
import time
import skimage
import math
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from skimage.color import grey2rgb,rgb2grey
from skimage.feature import hog
from skimage import io

# Custom scripts
import lscript as lsp
import modelling as md

In [3]:
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [5]:
height=112 
width=112 
channel=3
crop_size=112
cluster_length=16
feature_size=8192 #4096 #16384
baseline_feature_size=4608
baseline2_feature_size=2


nb_activities=4
nb_videos=5
nb_pixels_per_cell=(8,8)
nb_cells_per_block=(1,1)
nb_orientations=16
saved_path='/nobackup/leopauly/'
dataset_dir='/nobackup/leopauly/Activity_Analysis_new/Frames/Pushing/'

### Acitivity net

#### Defining model

In [ ]:
## Defining placeholders in tf for images and targets
x_image = tf.placeholder(tf.float32, [None, 16,height,width,channel],name='x') 

model_keras = md.C3D_ucf101_training_model_tf(summary=False)
out=model_keras(x_image)

#### Starting a session - Activity Net

In [ ]:
#### Start the session with logging placement.
sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))

### Restore model weights from previously saved model
saver = tf.train.Saver()
saver.restore(sess, os.path.join(saved_path,'activity_model.ckpt-67'))
print("Model restored from file: %s" % saved_path,flush=True)

In [6]:
## Extraction of features - Acitvity net
def extract_activitynet_video_features(vid):
    print('vid.shape',vid.shape)
    vid_=vid.reshape(-1,cluster_length,height,width,channel)
    f_v = sess.graph.get_tensor_by_name('flatten_1/Reshape:0')
    f_v_val=sess.run([f_v], feed_dict={'conv1_input:0':vid_,x_image:vid_,K.learning_phase(): 0 })
    features=np.reshape(f_v_val,(-1))
    return features

In [ ]:
def get_compress_frames_data(vid_folder, num_frames_per_clip=cluster_length):
  ''' Given a foler with frames from a video, fuction outputs a  num_frames_per_clip images which is the downsampled video'''

  ret_arr = []
  filenames=os.listdir(vid_folder)
  filenames = sorted(filenames)
  jump=math.floor((len(filenames)/num_frames_per_clip))
  loop=0
  for i in range(0,len(filenames),jump):
      if (loop>15):
        break
      if (filenames[i].endswith('.png')):
        image_name = str(vid_folder) + '/' + str(filenames[i])
        img = skimage.io.imread(image_name)  #Image.open
        img=np.array(img)
        img = cv2.resize(img,(crop_size,crop_size))
        img_data = np.array(img)
        ret_arr.append(img_data)
        loop=loop+1
  ret_arr=np.array(ret_arr)
  print('ret_arr',ret_arr.shape)
  return np.array(ret_arr)

In [7]:
## Extracting activity features from demo video
def activity_feature_extractor(folder):
    vid_folder=dataset_dir+folder
    print('Is the folder present:',os.path.isdir(vid_folder))
    temp_vid=get_compress_frames_data(vid_folder)
    print('Obtained video shape',temp_vid.shape)
    lsp.single_video_inline(temp_vid,0,cluster_length)
    activitynet_features=extract_activitynet_video_features(temp_vid)
    activitynet_features=np.array(activitynet_features)
    return activitynet_featuress

#### Rewards


In [8]:
## Demo
activitynet_features_demo=(activity_feature_extractor('V1_colour')) # Demo

Is the folder present: True


NameError: name 'get_compress_frames_data' is not defined

In [ ]:
## Same speed
activitynet_features=(activity_feature_extractor('V1')) 
distance=np.linalg.norm(activitynet_features_demo-activitynet_features)
print(distance)